In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import plotly.graph_objects as go
import numpy as np

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
def create_layer_viz(layer, x, y, name):
    return go.Scatter3d(
        x=x, y=y, z=np.zeros_like(x),
        mode='markers',
        marker=dict(size=5, color=layer.flatten(), colorscale='Viridis'),
        name=name
    )


In [ ]:
sample_input = np.random.rand(1, 28, 28, 1)
fig = go.Figure()

In [ ]:
x, y = np.meshgrid(range(28), range(28))
fig.add_trace(create_layer_viz(sample_input[0, :, :, 0], x.flatten(), y.flatten(), 'Input'))

In [ ]:
for i, layer in enumerate(model.layers):
    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.MaxPooling2D):
        output = layer(sample_input).numpy()
        h, w, c = output.shape[1:]
        x, y = np.meshgrid(range(w), range(h))
        for j in range(c):
            fig.add_trace(create_layer_viz(output[0, :, :, j], x.flatten() + (i+1)*30, y.flatten(), f'{layer.name}_{j}'))
    sample_input = layer(sample_input)


In [ ]:
fig.update_layout(
    scene=dict(
        xaxis_title='Width',
        yaxis_title='Height',
        zaxis_title='Activation'
    ),
    margin=dict(r=0, b=0, l=0, t=0),
    scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
)

fig.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
import tensorflow as tf

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, x_test.shape

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


((60000, 28, 28), (10000, 28, 28))

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
x_train = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000, 28, 28, 1))
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [ ]:
def create_model():
  inputs = keras.Input(shape=(28,28,1), name='input')
  x = layers.Conv2D(32, kernel_size = 3, strides=(1,1), padding='valid', activation='relu', name='conv_layer_1')(inputs)
  x = layers.MaxPooling2D(pool_size=2, name='max_pool1')(x)
  x = layers.Conv2D(64, kernel_size=3, strides=(1,1), padding='valid', activation='relu', name='conv_layer_2')(x)
  x = layers.MaxPooling2D(pool_size=2, name='max_pool2')(x)
  x = layers.Conv2D(128, kernel_size=3, strides=(1,1), padding='valid', activation='relu', name='conv_layer_3')(x)
  x = layers.Flatten(name='flatten')(x)
  x = layers.Dense(64, activation='relu', name='dense_1')(x)
  outputs = layers.Dense(10, activation='softmax', name='output')(x)

  model = keras.Model(inputs=inputs, outputs=outputs) # Define the model structure
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Compile the model with the desired configuration
  return model

In [ ]:
model = create_model()
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_layer_1 (Conv2D)                │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pool1 (MaxPooling2D)             │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_layer_2 (Conv2D)                │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pool2 (MaxPooling2D)             │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_layer_3 (Conv2D)                │ (None, 3, 3, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 167,114 (652.79 KB)

 Trainable params: 167,114 (652.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
modelcheckpoint = ModelCheckpoint(filepath='best_model.keras', save_best_only=True, monitor='val_loss', mode='min')
earlystopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')
my_model=model.fit(x_train, y_train, batch_size=32, epochs=1, validation_split=.2, callbacks=[modelcheckpoint])

1500/1500 ━━━━━━━━━━━━━━━━━━━━ 62s 40ms/step - accuracy: 0.8907 - loss: 0.3502 - val_accuracy: 0.9793 - val_loss: 0.0663
